In [1]:
import sys
sys.path.append("../code")
import pandas as pd
import random
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import blotto_utils as bu

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [11]:
pd.options.display.max_colwidth = 100

# 538 Data

In [7]:
weights = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

data_dir = '/Users/Alex/Documents/Alex/Yale/Fall 2019/CPSC_490/blotto/data/'

fivethirtyeight_data_dir = data_dir + 'fivethirtyeight/'

sorted(glob(fivethirtyeight_data_dir + '*'))

temp_df = pd.read_csv(glob(fivethirtyeight_data_dir + '*')[0])

new_column_names = list(map((lambda x : '_'.join(x.split(' ')).lower()), list(temp_df.columns)))
new_column_names[-1] = 'reason'

temp_df.rename(dict(zip(list(temp_df.columns), new_column_names)), axis=1, inplace=True)

temp_df['total_troops'] = temp_df['castle_1'] + \
                          temp_df['castle_2'] + \
                          temp_df['castle_3'] + \
                          temp_df['castle_4'] + \
                          temp_df['castle_5'] + \
                          temp_df['castle_6'] + \
                          temp_df['castle_7'] + \
                          temp_df['castle_8'] + \
                          temp_df['castle_9'] + \
                          temp_df['castle_10']

full_strats_df = temp_df[temp_df.total_troops == 100].copy()

full_strats_list = full_strats_df.to_dict('records')

simp_strats_list = [[int(x[1]) for x in list(y.items())[:10]] for y in full_strats_list]

## Divide into two players

In [10]:
random.seed(0)

In [11]:
a_strat_init = random.sample(simp_strats_list, 100)
b_strat_init = random.sample(simp_strats_list, 100)

In [12]:
len(a_strat_init), len(b_strat_init)

(100, 100)

# Agent